In [212]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam, SGD
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json

In [213]:
from model.input_fn import train_input_fn
from model.input_fn import test_input_fn
from model.model_fn import model_fn
from model.utils import Params

# 1. Define loss & base network
- 참고 소스 https://omoindrot.github.io/triplet-loss

In [9]:
#새로 적용하고자 하는 로스함수
def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.cast(mask_anchor_positive, dtype=tf.float32)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(input_tensor=anchor_positive_dist, axis=1, keepdims=True)
    tf.compat.v1.summary.scalar("hardest_positive_dist", tf.reduce_mean(input_tensor=hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.cast(mask_anchor_negative, dtype=tf.float32)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(input_tensor=pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(input_tensor=anchor_negative_dist, axis=1, keepdims=True)
    tf.compat.v1.summary.scalar("hardest_negative_dist", tf.reduce_mean(input_tensor=hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(input_tensor=triplet_loss)

    return triplet_loss

# 2. Load data

In [210]:
data_dir = '../imgs/img/'

In [226]:
BATCH_SIZE = 64
IMG_DIM = 28
NB_CLASSES = 5620

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                        data_dir,  # this is the target directory
                        target_size=(IMG_DIM, IMG_DIM),
                        batch_size=BATCH_SIZE)

Found 289219 images belonging to 5620 classes.


# 3. Create model & Train

In [258]:
def input_func_gen():
    ds = tf.data.Dataset.from_generator(lambda: train_generator,
                     output_types=(tf.float32, tf.float32),
                     output_shapes=([BATCH_SIZE, IMG_DIM, IMG_DIM, 3],
                                    [BATCH_SIZE]))
    it = tf.compat.v1.data.make_one_shot_iterator(ds)
    features, labels = it.get_next()
    labels = tf.math.argmax(labels, axis=1)
    return features, labels

In [259]:
input_func_gen()

(<tf.Tensor 'IteratorGetNext:0' shape=(64, 28, 28, 3) dtype=float32>,
 <tf.Tensor 'ArgMax:0' shape=() dtype=int64>)

In [260]:
input_func_gen

<function __main__.input_func_gen()>

In [261]:
model_dir = 'experiments/batch_hard'

json_path = os.path.join(model_dir, 'params.json')

params = Params(json_path)

In [262]:
K.clear_session()

In [263]:
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

In [264]:
tf.executing_eagerly()

False

In [265]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# Define the model
tf.compat.v1.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed=230,
                                model_dir=model_dir,
                                save_summary_steps=params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

# Train the model
tf.compat.v1.logging.info("Starting training for {} epoch(s).".format(params.num_epochs))
estimator.train(input_func_gen)

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Starting training for 20 epoch(s).
INFO:tensorflow:Calling model_fn.


ValueError: slice index 0 of dimension 0 out of bounds. for 'strided_slice' (op: 'StridedSlice') with input shapes: [0], [1], [1], [1] and with computed input tensors: input[1] = <0>, input[2] = <1>, input[3] = <1>.

In [182]:
num_channels = 32
channels = [num_channels, num_channels * 2]

In [172]:
for i, c in enumerate(channels):
    print(i, c)

0 32
1 64


In [ ]:
base_network.summary()    #base_network (즉 인코더) 서머리 보기
plot_model(base_network, to_file='./CNN/base_network.png', show_shapes=True, show_layer_names=True)    #base_network 도식화 + 저장

In [ ]:
# Evaluate the model on the test set
tf.compat.v1.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: test_input_fn(args.data_dir, params))
for key in res:
    print("{}: {}".format(key, res[key]))

In [ ]:
model.summary()     #전체 모델 서머리 보기
plot_model(model, to_file='./CNN/cnn_model.png', show_shapes=True, show_layer_names=True)     #전체 모델 도식화 + 저장

In [ ]:
# Training the model
H = model.fit([a, p, n], y_dummie, batch_size=50, epochs=10)

In [ ]:
#결과 확인
plt.plot(H.history['loss'])
plt.savefig('./CNN/loss_graph.png')
# plt.legend(['loss'], loc = 'upper left')
plt.show()

# 5. Save model & weights

In [ ]:
model_dir = './CNN/encoder.json'
model_weights_dir = './CNN/encoder.h5'

In [ ]:
#Save model
model_json = model.to_json()
with open(model_dir, "w") as json_file : 
    json_file.write(model_json)

#Save weights
model.save_weights(model_weights_dir)
print("Saved model to disk")